# Stock Forecasting using Transformers

In this notebook we implement a Transformer model to forecast stock data.

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/a/64438413

In [4]:
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras

## Time2Vec Embedding

https://arxiv.org/abs/1907.05321

In [43]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, embed_dim: int, activation: str = 'sin', **kwargs):
        """_summary_

        Args:
            embed_dim (int): Length of the time embedding vector.
            activation (str, optional): Periodic activation function. Possible values are ['sin', 'cos']. Defaults to 'sin'.
        """
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.activation = activation.lower() # Convert to lower-case.

        # Set periodic activation function.
        if self.activation.startswith('sin'):
            self.activation_func = tf.sin
        elif self.activation.startswith('cos'):
            self.activation_func = tf.cos
        else:
            raise ValueError(f'Unsupported periodic activation function "{activation}"')

    def build(self, input_shape: list[int]):

        # Weight and bias term for linear portion (i = 0)
        # of embedding.
        self.w_linear = self.add_weight(
            name='w_linear',
            shape=(input_shape[1], 1,),
            initializer='uniform',
            trainable=True,
        )
        self.b_linear = self.add_weight(
            name='b_linear',
            shape=(input_shape[1], 1,),
            initializer='uniform',
            trainable=True,
        )

        # Weight and bias terms for the periodic
        # portion (1 <= i <= k) of embedding.
        self.w_periodic = self.add_weight(
            name='w_periodic',
            shape=(input_shape[-1], self.embed_dim,),
            initializer='uniform',
            trainable=True,
        )
        self.b_periodic = self.add_weight(
            name='b_periodic',
            shape=(input_shape[1], self.embed_dim,),
            initializer='uniform',
            trainable=True,
        )

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """_summary_

        Args:
            x (tf.Tensor): Input tensor with shape (batch_size, sequence_length, feature_size)

        Returns:
            tf.Tensor: Output tensor with shape (batch_size, sequence_length, embed_dim + 1)
        """

        # Linear term (i = 0).
        embed_linear = tf.tensordot(x, self.w_linear, axes=1) + self.b_linear

        # Periodic terms (1 <= i <= k).
        inner = tf.tensordot(x, self.w_periodic, axes=1) + self.b_periodic
        embed_periodic = self.activation_func(inner)

        # Return concatenated linear and periodic features.
        return tf.concat([embed_linear, embed_periodic], axis=-1)

    def get_config(self) -> dict:
        """Retreive custom layer configuration for future loading.

        Returns:
            dict: Configuration dictionary.
        """
        config = super().get_config().copy()
        config.update({
            'embed_dim': self.embed_dim,
            'activation': self.activation,
        })
        return config

stock_feat = 2
seq_len = 128
embed_dim = 32
inp = keras.Input(shape=(seq_len, stock_feat))
print(f"{inp.shape=}")
x = Time2Vec(embed_dim)(inp)
print(f"{x.shape=}")
x = keras.layers.Concatenate(axis=-1)([inp, x])
print(f"{x.shape=}")

inp.shape=TensorShape([None, 128, 2])
x.shape=TensorShape([None, 128, 33])
x.shape=TensorShape([None, 128, 35])
